# Bayesian peak assignment #

This script is used to perform Bayesian inference to determine what are possible assignments of theoretical DFT vibrational frequency calculations to experimentally observed IR absorption peaks.

It requires that an excel database with the DFT-calculated frequencies and an IR spectrum text dump is available, for formatting, see below (where the images of excel spreadsheets are).

The complexity of IR spectra make it difficult to associate with theoretical frequency calculations, as we expect that the DFT-calculation will predict a frequency within a range of deviation from the real, experimental peak. While it can be an easier case for molecules such as Vanillin, when a lot of peaks are involved in a smaller space (such as Syringaldehyde), or the spectral peaks are broadened (such as Hydroxybenzaldehyde), it quickly becomes a difficult and uncertain problem.

While it's not going to lead to an exact answer, a Sudoku-esque approach can be taken to make some sense out of the spectral pattern, using deduction based on clues such as "if theoretical frequency "x" goes to experimental peak "x", then theoretical frequency "y" doesn't fit anywhere". 

This is the kind of logic that this script and approach are attempting to utilise to present options that make sense in terms of assignment. While a metric is included and called "Probability", it essentially indicates how good the fit is, and it is very possible that the chosen answer won't be the "most likely", as other clues such as intensity and chemical sense are applied.

An assumption applied here is that ALL theoretical frequencies chosen will be represented experimentally, and as such only identifiable and trusted vibrational modes are chosen. While it might sound like bad practise to hold the theoretical frequencies as indicative of what the experimental peaks should be, the opposite cannot be applied, as the experimental peaks will always be more than the theoretical frequencies, due to phenomena that DFT cannot account for (combination bands, Fermi resonance, higher state vibrations beyond the fundamental, phenomena such as hydrogen bonding, etc...).

Importing the required libraries.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

Defining functions that will be used later on for the inference:

1. Gaussian:
Defines the equation for a Gaussian curve. 

$$
      f(\chi) = \frac{1}{\sqrt{2 \pi \sigma^{2}}}e^{\frac{({\chi-\mu})^{2}}{2 {\sigma^{2}}}}
$$
* $\chi$ refers to a specific point on the x-axis. 
* $f(\chi)$ refers to the y-value for the specific point. 
* $\mu$ refers to the distribution's mean value.
* $\sigma$ refers to the distribution's standard deviation.



2. Bayes:
Defines the equation for Bayes' theorem. 

$$
    P(H|E) = \frac{P(E|H)P(H)}{P(E)}
$$
* $P(H|E)$ refers to the posterior probability (i.e. the probability that a hypothesis "H" holds true when evidence "E" is considered).
* $P(E|H)$ refers to the likelihood probability (i.e. the probability fo evidence "E" being observed assuming that hypothesis "H" is true).
* $P(H)$ refers to the prior probability (i.e. the current belief probability for hypothesis "H" being true. The posterior calculated from one piece of evidence can then be used as the prior probability when presenting the next piece of evidence).
* $P(E)$ refers to the marginal probability (i.e. the full probability space for explaining evidence "E", in other words, the product of the likelihoods and prior probabilities for all hypotheses with the current piece of evidence:

$$
    P(E) = P(E|H_{1})P(H_{1}) * P(E|H_{2})P(H_{2}) * P(E|H_{3})P(H_{3}) * ... * P(E|H_{n})P(H_{n})
$$


3. Bayes_models:
Defines the comparative version of Bayes' theorem between two hypotheses.

$$
    \frac{P(H_{1}|E)}{P(H_{2}|E)} = \frac{P(E|H_{1})}{P(E|H_{2})} \frac{P(H_{1})}{P(H_{2})}
$$

* This form of Bayes' theorem results in a ratio (Bayes' factor) which indicates the strength of hypothesis 1 over hypothesis 2 based on the presented evidence. Taking the $log_{10}$ of the Bayes' factor can be used to approximate the odds between the two hypotheses (Jeffreys' scale).

4. STD_conv:
Converts STD from a percentage (which is used in the current work) to a corresponding numerical value based on the investigated experimental peak's value. A "unique" STD for each peak only makes sense because we know that errors scale with the peak frequency, therefore a STD as a percentage of the peak makes sense and ensures that all distributions will have a similar "width".

$$
    STD = (\chi_{Exp} + (\chi_{Exp} * \frac{MSD_{(\%)}}{100})) * \frac{STD_{(\%)}}{100}
$$

5. Mean_conv:
Converts MSD from a percentage (which is used in the current work) to a corresponding numerical value and adds onto the experimental peak. As the MSD is conceptualised as the mean deviation from an experimental value (i.e. how higher or lower from the experimental value a theoretical value will predict on average), then it can be used as the expected value in a probability distribution, which is to say, the "peak" of the distribution. As such, this function finds that peak value.

$$
    MSD = \chi_{Exp} + (\chi_{Exp} * \frac{MSD_{(\%)}}{100}))
$$

In [14]:
def Gaussian(x, std, mean):
    return (1 / (math.sqrt(2 * math.pi * (std**2)))) * (math.exp(- ((x-mean) ** 2) / (2 * (std ** 2))))

def Bayes(Prior, Likelihood, Marginal):
    Posterior = (Likelihood * Prior) / Marginal
    return Posterior

def Bayes_models(Likelihood_Hi, Prior_Hi, Likelihood_Hj, Prior_Hj):
    Factor = (Likelihood_Hi / Likelihood_Hj) * (Prior_Hi / Prior_Hj)
    return Factor

def STD_conv(peak, rel_MSD, rel_STD):
    STD = (peak + (peak * (rel_MSD / 100))) * (rel_STD / 100)
    return STD

def Mean_conv(peak, rel_MSD):
    Mean = peak + (peak * (rel_MSD / 100))
    return Mean

In [3]:
STD_conv(1663, -0.18, 1.18)

19.58807788

This section defines the molecules that will be studied, and the path to the excel database.

1. Molecule_list: 
Insert the molecules as strings (e.g. "Vanillin"). These MUST have corresponding sheets in the spreadsheet with the molecule spelled as defined and with the endings "-DFT" and "-IR" for the DFT frequencies and IR spectrum, respectively. (e.g. Vanillin-DFT and Vanillin-IR).
2. path:
Insert location of excel spreadsheet database, for python to read locations, "\" slashes must be replaced by either "/" or "\\", or an r must be written before the location string (e.g. 'C:\\User\\Location\\Database.xlsx', 'C:/User/Location/Database.xlsx' or r'C:\User\Location\Database.xlsx').
The excel sheets for the DFT frequencies and IR spectrum must be in the form shown below:

![alt text](Excel_DFT.png "DFT frequencies")

* "Scaled frequency" refers to the "Unscaled Frequency" after it has been multipled by the "Scaling factor". 
* "Normal mode" is hand-written by the author, while it's intended to record the important vibrational modes of the vibrations, in essence, the script only cares about something being written there. These indicate the theoretical frequencies that will be extracted by the script for inference.
* "Normal mode (Not computed for peak assignment) is the same as "Normal mode", except the script won't extract values if they don't have an entry written in "Normal mode", this is intended as a record for modes that might be important but not appear or appear significantly in the IR spectrum. These won't be extracted and used in the script given that the "Normal mode" entry is empty.

![alt text](Excel_IR.png "IR frequencies")

* "Frequency" and "Intensity" can be easily extracted as such from an IR spectrometer.
* "Difference" refers to the difference between that row's intensity with the previous row's intensity (e.g. for "C3": = B3 - B2). This can be just written for one cell and then dragged down to apply the formula for all rows.
* "Sign" refers to a measure of the sign of the change, formulated as the product of this row's difference with the next row's difference (e.g. for "D3": = C3 * C4 * 1000000), the 1000000 factor is just there to make the resulting number not show a lot of decimal places. This tells the script that there is a change in the spectrum (peak or trough) and whether or the change corresponds to a peak (negative value) or trough (positive value). Again, just including this value and dragging the cell will create the entry for all cells.
* "Observation" allows for manual assignment or dis-assignment of peaks. While the script will automatically detect all peaks and include them (that is, a point which shows a change from a higher to a lower intensity), it can also include noise, which might be wrongly labelled in the assignment. On the other hand, it's possible that the script might miss a peak because it forms a shoulder to a close-by peak instead of a classically defined individual peak. This column allows the user to manually exclude a peak from the script by including the entry "Noise" in the row of the included peak, or manually include a peak into the script by including the entry "Shoulder" in the row that best represents the shoulder.

In [15]:
Molecule_list = ["Vanillin"]
path = 'Vanillin_results_test.xlsx'

for Molecule in Molecule_list:
    
    df_th = pd.read_excel(path, str(Molecule) + "-DFT").fillna("")
    df_exp = pd.read_excel(path, str(Molecule) + "-IR").fillna("")

FileNotFoundError: [Errno 2] No such file or directory: 'Vanillin_results_test.xlsx'

In this section, the user adds the deviation metrics that represent the certainty of the theoretical method. Particularly the Mean Signed Deviation (MSD) and Standard Deviation (STD) as percentages. Fixed value MSD and STD can also be used if needed, but keep in mind that a different option will need to be selected in the script down the line.

In this work, the MSD and STD used for PBE0/6-311++G(2d,2p) vibrational frequencies, scaled by a factor of 0.950, were determined through vanillin by identifying and assigning characteristic peaks by eye (In the case of vanillin, this was feasible due to the clean spectrum and well defined peaks that could be easily assigned). See supporting info (S4 for equations, S12 for data and calculation of metrics and S13 for "universal" scaling factor optimisation for PBE0/6-311++G(2d,2p).).

A "Threshold" value will need to be included (recommended: 50). This tells the script how far apart a theoretical frequency can be from an experimental value before it stops being considered. This is a way for the script to avoid forming all possible theoretical-to-experimental assignments possible, as those expand to astronomical amounts. Instead, the script simply takes a theoretical frequency and only forms assignments with experimental peaks within the given "Threshold" value in cm$^{-1}$.

As this script is intended for batch processing of multiple molecules in sequence, an "If" statement is included in the case the user wants to allow for a higher, or lower, threshold for one specific molecule (In our case, Umbelliferone was given a threshold of 70 due to its highly off-set spectrum). To use that, simply specify the molecule in the "if" line and the threshold to be used. If more than one molecules need to have unique thresholds, an "elif" statement can be added for each new molecule.

In [5]:
    MSD = -0.12
    STD = 1.18

    if Molecule == "Vanillin":
        Threshold = 50
    elif Molecule == "Vanillin":
        Threshold = 50
    else:
        Threshold = 50

This section is where the script reads the data provided and creates its own variable lists for all theoretical and all experimental peaks given to it. The only things that need to be provided here are the range within the experimental IR spectrum for the script to scan for peaks. Again, some flexibility is provided by allowing different ranges to be included for different molecules (e.g. in this work, Cinnamaldehyde was found to have C-H wagging peaks at less than 700 cm$^{-1}$, but expanding this range for all molecules would have made the inference take much longer).

In this part, the user can change the number ranges in the loop to change the range for a molecule (e.g. within the "If Molecule == "Vanillin":" loop, the values in both "if df_exp.loc..." and "elif str(df_exp.loc..." lines (680 and 1800) indicate a scan range from 680 to 1800 cm$^{-1}$. Changing those will change the range appropriately.

In [6]:
    print("Performing inference for " + str(Molecule) + ".")
    
    Theoretical_peaks = []
    Experimental_peaks = []
    
    if Molecule == "Vanillin":
        for mode in range(0, len(df_th.loc[:, "Mode"])):
            if str(df_th.loc[mode, "Normal mode"]) != "":
                Theoretical_peaks.append(round(df_th.loc[mode, "Scaled Frequency"]))
        
        for wavenumber in range(0, len(df_exp.loc[:, "Frequency"])):
            if df_exp.loc[wavenumber, "Sign"] < 0 and df_exp.loc[wavenumber, "Difference"] > 0 and df_exp.loc[wavenumber, "Frequency"] > 680 and df_exp.loc[wavenumber, "Frequency"] < 1650 and str(df_exp.loc[wavenumber, "Observation"]) != "Noise":
                Experimental_peaks.append(round(df_exp.loc[wavenumber, "Frequency"]))
            elif str(df_exp.loc[wavenumber, "Observation"]) == "Shoulder" and df_exp.loc[wavenumber, "Frequency"] > 680 and df_exp.loc[wavenumber, "Frequency"] < 1800:
                Experimental_peaks.append(round(df_exp.loc[wavenumber, "Frequency"]))
                
    elif Molecule == "Vanillin":
        for mode in range(0, len(df_th.loc[:, "Mode"])):
            if str(df_th.loc[mode, "Normal mode"]) != "":
                Theoretical_peaks.append(round(df_th.loc[mode, "Scaled Frequency"]))
        
        for wavenumber in range(0, len(df_exp.loc[:, "Frequency"])):
            if df_exp.loc[wavenumber, "Sign"] < 0 and df_exp.loc[wavenumber, "Difference"] > 0 and df_exp.loc[wavenumber, "Frequency"] > 740 and df_exp.loc[wavenumber, "Frequency"] < 1650 and str(df_exp.loc[wavenumber, "Observation"]) != "Noise":
                Experimental_peaks.append(round(df_exp.loc[wavenumber, "Frequency"]))
            elif str(df_exp.loc[wavenumber, "Observation"]) == "Shoulder" and df_exp.loc[wavenumber, "Frequency"] > 740 and df_exp.loc[wavenumber, "Frequency"] < 1800:
                Experimental_peaks.append(round(df_exp.loc[wavenumber, "Frequency"]))
                
    else:
        for mode in range(0, len(df_th.loc[:, "Mode"])):
            if str(df_th.loc[mode, "Normal mode"]) != "":
                Theoretical_peaks.append(round(df_th.loc[mode, "Scaled Frequency"]))
        
        for wavenumber in range(0, len(df_exp.loc[:, "Frequency"])):
            if df_exp.loc[wavenumber, "Sign"] < 0 and df_exp.loc[wavenumber, "Difference"] > 0 and df_exp.loc[wavenumber, "Frequency"] > 750 and df_exp.loc[wavenumber, "Frequency"] < 1650 and str(df_exp.loc[wavenumber, "Observation"]) != "Noise":
                Experimental_peaks.append(round(df_exp.loc[wavenumber, "Frequency"]))
            elif str(df_exp.loc[wavenumber, "Observation"]) == "Shoulder" and df_exp.loc[wavenumber, "Frequency"] > 750 and df_exp.loc[wavenumber, "Frequency"] < 1800:
                Experimental_peaks.append(round(df_exp.loc[wavenumber, "Frequency"]))

Performing inference for Vanillin.


As the algorithm initially iterated through all possible "hypotheses", taken as all the possible assignments of theoretical to experimental peaks (which is to say all the permutations of n-experimental peaks and r-theoretical peaks), it was useful to build in a small segment that calculates how many hypotheses are formed with a given set.

$$
P(n,r) = \frac{n!}{(n-r)!}
$$    

This had proven to be useful after realising the silliness of inputting 14 theoretical peaks and 21 experimental peaks in this case, as it can easily lead to massive computational times.

This segment simply prints out how many permutations are possible (i.e. would be formed without the threshold limitation).

In [7]:
    Hypotheses_number = math.factorial(len(Experimental_peaks)) / (math.factorial(len(Experimental_peaks) - len(Theoretical_peaks)))
    print("This combination of peak sets will result in " + str(int(Hypotheses_number)) + " hypotheses.") 

This combination of peak sets will result in 10137091700736000 hypotheses.


Obviously, this is a computational road-block if we want to consider all peaks within the region of interest, so, a loop was coded to build the permutations manually (instead of using available packages), while excluding assignments above the threshold as they appear. This has proven to greatly reduce the number of hypotheses, as removing permutations during the process automatically removes all permutations that would be built on top of that.
 
In this case, the 1E16 possible hypotheses are reduced to a much more manageable 2016, simply by filtering claims which would be too unlikely, which still take a bit of time to process, but are certainly doable.

In [8]:
    Theo = list(range(1, len(Theoretical_peaks)+1))
    Exp = list(range(1, len(Experimental_peaks)+1))    
    
    Hypotheses = []
    
    temp_hypothesis = []
    
    exec("Unassigned_peaks_" + Molecule + " = []")
    Theo_to_remove = []
    
    for i in Theo:
        counter = 0
        temp_hypothesis_2 = []
        if i == 1:
            for j in Exp:
                if abs(Theoretical_peaks[i-1] - Experimental_peaks[j-1]) < Threshold:
                    temp_hypothesis.append([j])     
        else:
            for h in temp_hypothesis:
                for j in Exp:
                    temp_h = []
                    if abs(Theoretical_peaks[i-1] - Experimental_peaks[j-1]) < Threshold and j not in h:
                        temp_h = h[:]
                        temp_h.append(j)
                        temp_hypothesis_2.append(temp_h)
            if temp_hypothesis_2 != []:
                temp_hypothesis = temp_hypothesis_2[:]
            elif temp_hypothesis_2 == []:
                print("Could not fit an assignment for theoretical peak: " + str(Theoretical_peaks[i-1]))
                Theo_to_remove.append(i)    
                exec("Unassigned_peaks_" + Molecule + ".append(Theoretical_peaks[i-1])")
    
    exec("[Theoretical_peaks.remove(i) for i in Unassigned_peaks_" + Molecule + "]")
    [Theo.remove(i) for i in Theo_to_remove]
    
    Hypotheses = temp_hypothesis[:]    
    
    print(str(int(Hypotheses_number - int(len(Hypotheses)))) + " hypotheses have been filtered, " + str(len(Hypotheses)) + " remaining.")
    
    Hypotheses_labelled = [] 
    Evidence_labelled = []

10137091700733984 hypotheses have been filtered, 2016 remaining.


This section creates an appropriately sized processing dataframe, which is to say, a table of H rows (equal to the number of Hypotheses) and E columns (equal to the number of Evidence provided).

To specify the connection:

1. Hypotheses in this case are considered to be all possible complete sets of assignments of theoretical frequencies to experimental peaks, e.g.: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] is an example hypothesis here, which specifies that theoretical frequency number 1 is assigned to Experimental peak number 2 (The first value indicates that it's the experimental value that we assign to the FIRST theoretical frequency, in this case, experimental peak number 2). As there are 2016 possible sets (i.e. unique sets in which all theoretical frequencies are assigned to different experimental peaks within the threshold range), we assume that there are 2016 possible hypotheses that need to be tested.


2. Evidence in this case is presented as each theoretical frequency. This sounds a bit counter-intuitive and is easy to mis-interpret. 

    * As an analogy, consider a problem where we have 5 keys and 3 locks, we know that 3 of those keys will belong to the 3 locks. In this case, if we try a key on lock 1 and find that it opens it, then the probability of key 2 belonging to lock 2 would be affected, as now we know that we have 4 keys and 2 locks. This is analogous to saying that we have 21 detected experimental peaks and 14 theoretical peaks, assuming that all 14 theoretical peaks must be peaks that can be detected experimentally (from DFT-calculation and chemical knowledge). Therefore, assigning as an example theoretical frequency 1 to experimental peak 1 affects the probability of all other peaks. The difference here is that we are 100% sure that key 1 opens lock 1 in this scenario, this wouldn't be the case for our peaks, but instead, we use a probability (a measure of how well the theoretical frequency fits to the experimental peak, see image below). This means that the final inference for each hypothesis will be a product of well all the theoretical frequencies fit to their assigned experimental peaks.
    
    * Perhaps the most significant issue this method deals with is considering all other peak assignments as well as the one in the current iteration. This means that not just the closeness of a frequency to a peak is taken into consideration, but also how well the rest of the frequencies will fit with other peaks if this assignment is assumed.
    
    ![alt text](Peak_distribution.jpg "Peak probability distribution")
    
    In this case, a "probability distribution function" for Vanillin's experimental 1298 cm$^{-1}$ peak is shown, after the expected value (mean) has been shifted by the MSD. The "mean" value represents the most likely frequency, and how "likely" other frequencies are to be calculated as fall within the distribution, in this case, a theoretical frequency of 1274 cm${^-1}$ was selected. The image represents a visual representation of this concept. 
    


In [9]:
    for i in range(1,len(Hypotheses)+1):
        Hypotheses_labelled.append("Hypothesis " + str(i))
    
    for i in range(1,len(Theoretical_peaks)+1):
        Evidence_labelled.append("E"+ str(i))
    
    Probs = pd.DataFrame(columns = Evidence_labelled, index = Hypotheses_labelled)
    Probs["Theoretical peak assignment"] = Hypotheses
    
    Peaks1 = pd.DataFrame({"Experimental peak label": Exp})
    Peaks2 = pd.DataFrame({"Experimental peak": Experimental_peaks})
    Peaks3 = pd.DataFrame({"Theoretical peak label": Theo})
    Peaks4 = pd.DataFrame({"Theoretical peak": Theoretical_peaks})
    Peaks = pd.concat([Peaks1, Peaks2, Peaks3, Peaks4], axis = 1)

    Probs

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14,Theoretical peak assignment
Hypothesis 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1..."
Hypothesis 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1..."
Hypothesis 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 2..."
Hypothesis 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 1..."
Hypothesis 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hypothesis 2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[4, 5, 8, 11, 9, 10, 13, 12, 14, 17, 15, 19, 2..."
Hypothesis 2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[4, 5, 8, 11, 9, 10, 13, 12, 14, 17, 16, 18, 1..."
Hypothesis 2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[4, 5, 8, 11, 9, 10, 13, 12, 14, 17, 16, 18, 2..."
Hypothesis 2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[4, 5, 8, 11, 9, 10, 13, 12, 14, 17, 16, 19, 1..."


This section is now the Bayesian processing loop, during which for each hypothesis, Bayes' theorem (as defined earlier) is applied, taking the likelihood as the y-value for a gaussian probability distribution, at an x-value (the theoretical frequency), centered at the assigned experimental peak affected by the MSD with a spread defined by the STD. This only works with the MSD and STD represented as percentages.

In [10]:
    for E in range(0, len(Theoretical_peaks)):
        counter = 0
        if E == 0:
            for H in Hypotheses:
                Marginal = []
                for i in Hypotheses:   
                    Marginal.append(Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[i[E]-1], MSD, STD), Mean_conv(Experimental_peaks[i[E]-1], MSD)) * (1/len(Hypotheses)))
                Marginal = sum(Marginal)
                P_H_given_E = Bayes((1/len(Hypotheses)), Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[H[E]-1], MSD, STD), Mean_conv(Experimental_peaks[H[E]-1], MSD)), Marginal)
                Probs.iloc[counter, E] = P_H_given_E
                counter = counter + 1
        else:
            for H in Hypotheses:
                Marginal = []
                counter_2 = 0
                for i in Hypotheses:   
                    Marginal.append(Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[i[E]-1], MSD, STD), Mean_conv(Experimental_peaks[i[E]-1], MSD)) * Probs.iloc[counter_2, E-1])
                    counter_2 = counter_2 + 1
                Marginal = sum(Marginal)
                P_H_given_E = Bayes(Probs.iloc[counter, E-1], Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[H[E]-1], MSD, STD), Mean_conv(Experimental_peaks[H[E]-1], MSD)), Marginal)
                Probs.iloc[counter, E] = P_H_given_E
                counter = counter + 1

An alternative option is provided below for fixed MSD and STD values. Simply unhash and use instead of the previous option in this case.

In [11]:
    # for E in range(0, len(Theoretical_peaks)):
    #     counter = 0
    #     if E == 0:
    #         for H in Hypotheses:
    #             Marginal = []
    #             for i in Hypotheses:   
    #                 Marginal.append(Gaussian(Theoretical_peaks[E], STD, MSD + Experimental_peaks[E]) * (1/len(Hypotheses)))
    #             Marginal = sum(Marginal)
    #             P_H_given_E = Bayes((1/len(Hypotheses)), Gaussian(Theoretical_peaks[E], STD, MSD + Experimental_peaks[E]), Marginal)
    #             Probs.iloc[counter, E] = P_H_given_E
    #             counter = counter + 1
    #     else:
    #         for H in Hypotheses:
    #             Marginal = []
    #             counter_2 = 0
    #             for i in Hypotheses:   
    #                 Marginal.append(Gaussian(Theoretical_peaks[E], STD, MSD + Experimental_peaks[E]) * Probs.iloc[counter_2, E-1])
    #                 counter_2 = counter_2 + 1
    #             Marginal = sum(Marginal)
    #             P_H_given_E = Bayes(Probs.iloc[counter, E-1], Gaussian(Theoretical_peaks[E], STD, MSD + Experimental_peaks[E]), Marginal)
    #             Probs.iloc[counter, E] = P_H_given_E
    #             counter = counter + 1

This section performs the model comparison version of Bayes' theorem for only the 5 highest probability hypotheses.

In [12]:
    Probs = Probs.sort_values(Probs.columns[-2], ascending = False)
    
    Hypotheses = Probs.iloc[0:5, -1].tolist()
    temp_H = []
    for Hypothesis in Hypotheses:
        exec("temp_H.append(" +str(Hypothesis)+ ")")
    Hypotheses = temp_H
    
    Probs_models = pd.DataFrame(columns = range(1, len(Hypotheses)+1), index = range(1, len(Hypotheses)+1)).astype(float)
    Probs_models_log = pd.DataFrame(columns = range(1, len(Hypotheses)+1), index = range(1, len(Hypotheses)+1)).astype(float)
    
    #Relative MSD and STD
    for E in range(0, len(Theoretical_peaks)):
        if E == 0:
            for Hi in range(0, len(Hypotheses)):
                for Hj in range(0, len(Hypotheses)):
                    if Hj == 0:
                        Probs_models.iloc[Hi, Hj] = 1
                    else:
                        Probs_models.iloc[Hi, Hj] = Bayes_models(
                            Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[Hypotheses[Hi][E]-1], MSD, STD), Mean_conv(Experimental_peaks[Hypotheses[Hi][E]-1], MSD)),
                            (1/len(Hypotheses)),
                            Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[Hypotheses[Hj][E]-1], MSD, STD), Mean_conv(Experimental_peaks[Hypotheses[Hj][E]-1], MSD)),
                            (1/len(Hypotheses))
                        )
                        
        else:
            for Hi in range(0, len(Hypotheses)):
                for Hj in range(0, len(Hypotheses)):
                    if Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[Hypotheses[Hj][E]-1], MSD, STD), Mean_conv(Experimental_peaks[Hypotheses[Hj][E]-1], MSD)) == 0 or Probs.iloc[Hj, E-1] == 0:
                        Probs_models.iloc[Hi, Hj] = 1
                    else:
                        Probs_models.iloc[Hi, Hj] = Bayes_models(
                            Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[Hypotheses[Hi][E]-1], MSD, STD), Mean_conv(Experimental_peaks[Hypotheses[Hi][E]-1], MSD)),
                            Probs.iloc[Hi, E-1],
                            Gaussian(Theoretical_peaks[E], STD_conv(Experimental_peaks[Hypotheses[Hj][E]-1], MSD, STD), Mean_conv(Experimental_peaks[Hypotheses[Hj][E]-1], MSD)),
                            Probs.iloc[Hj, E-1]
                        )
    Probs_models_log = Probs_models.applymap(math.log10)
    for Hypothesis in range(1, len(Probs_models.index)+1):
        print("Lowest Hypothesis " + str(Hypothesis) + " factor is " + str(min(Probs_models.iloc[Hypothesis-1, :])))
    

Lowest Hypothesis 1 factor is 1.0
Lowest Hypothesis 2 factor is 0.8786532020206881
Lowest Hypothesis 3 factor is 0.6565819876497873
Lowest Hypothesis 4 factor is 0.5769078658375936
Lowest Hypothesis 5 factor is 0.2692290991953564


Finally, this section saves the results in a new excel spreadsheet.

The spreadsheet contains 4 sheets:
1. Legend: Shows all considered theoretical frequencies and experimental peaks, their indexed label and their frequency.
2. Probabilites: Shows all hypotheses and their probabilities after every round of evidence (order in descending form in terms of probability of final evidence).
3. Model comparisons: Shows the Bayesian factors between the five highest probability hypotheses.
4. Model comparsions (log): Shows the $log_{10}$ of the Bayesian factor between the five highest probability hypotheses.

In [13]:
    with pd.ExcelWriter('IR_assignments_' + Molecule + '.xlsx') as writer:
        Peaks.to_excel(writer, sheet_name= "Legend")
        Probs.to_excel(writer, sheet_name='Probabilities')
        Probs_models.to_excel(writer, sheet_name='Model comparisons')
        Probs_models_log.to_excel(writer, sheet_name='Model comparisons (log)')